In [ ]:
# !pwd
import os
import pandas as pd
import numpy as np

In [ ]:
all_sub_adjMat = np.load("adni_612_corrmat.npy")
sub_names = np.load("adni_612_filenames.npy", allow_pickle= True)
print(len(all_sub_adjMat), len(sub_names))

In [ ]:
len(all_sub_adjMat), np.shape(all_sub_adjMat), type(all_sub_adjMat)

In [ ]:
all_sub_adjMat = np.delete(all_sub_adjMat, (130,460, 540), axis=0)
sub_names = np.delete(sub_names, (130,460, 540))
print(len(all_sub_adjMat), len(sub_names))

In [ ]:
with open('ADNI_largerdataset_labels.csv') as f:
    content = f.readlines()
content = [x.strip() for x in content]
content.pop(-1)
print(content[0])
print(len(content))

In [ ]:
with open('imageid_visitid.txt', 'r') as g:
    lines = g.readlines()
lines = [x.strip() for x in lines]
image_ids = [x[0:7] for x in lines]
visit_ids =  [x[9:] for x in lines]
print(lines[0], image_ids[0], visit_ids[0])

In [ ]:
df = pd.read_csv('ADNI_largerdataset_labels.csv') # can also index sheet by name or fetch all sheets
label = df['Group'].tolist()
subject = df['Image Data ID'].tolist()
print(len(label),label[:5], subject[:5])

In [ ]:
visitnames = [str(i)[11:] for i in sub_names]
visitnames = [str(i)[0:-21] for i in visitnames]
print(visitnames[0])

subnames = []
for i in range(len(visitnames)):
    index = visit_ids.index(visitnames[i])
    subnames.append(image_ids[index])
    

In [ ]:
print(len(subnames), subnames[:5])

In [ ]:
subnames.index('visitid')

In [ ]:
print(len(subnames), subnames[:5])
for x,i in enumerate(subnames):
    print(x,i)
    break
    
# print(len(subject), subject[])

In [ ]:
adni90_feature = []
adni90_labels = []
adni90_adjmat = []
edge_fea = []

for x,i in enumerate(subnames):
    if i =='visitid':
        continue
    ind = subject.index(i)
    adni90_labels.append(label[ind])
    adni90_feature.append(all_sub_adjMat[x])

for i in adni90_feature:
    temp = np.zeros((90,90))
    for x in range(90):
        for y in range(90):
            if abs(i[x][y]) > 0.15:
                temp[x][y] = 1
                edge_fea.append(i[x][y])
            else:
                edge_fea.append(-999)
    adni90_adjmat.append(list(temp))
            
    
print(len(adni90_labels), len(adni90_adjmat), len(adni90_feature))
print(np.array(adni90_adjmat[0]).shape)

In [ ]:
# # choose the classification type

lab1 = []
exp1 = []
fea1 = []

c1,c2 = 0, 0

for c,i in enumerate(adni90_labels):
    if i == 'EMCI' or i == 'LMCI' or i == 'MCI':
#     if i == 'CN':
        c1 += 1
#         if c1 == 150:
#             c1 -= 1
#             continue
        lab1.append(1)
        exp1.append(adni90_adjmat[c])
        fea1.append(adni90_feature[c])
        
#     elif i == 'AD':
    elif i == 'CN':
        c2 += 1
        lab1.append(2)
        exp1.append(adni90_adjmat[c])
        fea1.append(adni90_feature[c])
    

print(len(exp1), len(fea1), len(lab1))
print(c1, c2) 

In [ ]:
# print(len(edge_fea), len(adni_a))
print(lab1)

In [ ]:
!pwd
os.chdir("/home/anoopkumar/Brain/HGP-SL/data")
!pwd

In [ ]:
# for ADNI_A.txt

edgeFea = []
count = -1
adni_a = []
for i, x in enumerate(exp1):
    temp = []
    for a in range(len(x)):
        for b in range(len(x[0])):
            count += 1
            if a < b and x[a][b] == 1:
                temp.append((a+1+i*90,b+1+i*90))
                edgeFea.append(edge_fea[count])

    adni_a.extend(temp)

print(len(adni_a))
print(adni_a[-10:])


with open("ADNI_612/raw/ADNI_612_A.txt","w") as f:
    for item in adni_a:
        f.write("%s, %s\n" % (item[0],item[1]))

print(len(edgeFea))

In [ ]:
# ADNI_graph_indicator.txt

node_graph_indicator = []
for i in range(1,len(exp1)+1):
    node_graph_indicator.extend([i for j in range(90)])

with open("ADNI_612/raw/ADNI_612_graph_indicator.txt","w") as f:
    for item in node_graph_indicator:
        f.write("%s\n" % item) 

In [ ]:
# ADNI_graph_labels.txt

with open("ADNI_612/raw/ADNI_612_graph_labels.txt","w") as f:
    for item in lab1:
        f.write("%s\n" % item) 

In [ ]:
# ADNI node labels.txt



node_labels = []

for i in range(len(exp1)):
    for j in range(90):
        node_labels.extend([j])
print(len(node_labels), len(node_graph_indicator))                   
with open("ADNI_612/raw/ADNI_612_node_labels.txt","w") as f:
    for item in node_labels:
        f.write("%s\n" % item) 

In [ ]:
# ADNI edge attr


with open("ADNI_612/raw/ADNI_612_edge_attributes.txt","w") as f:
    for item in edgeFea:
        f.write("%s\n" % item) 

In [ ]:
import torch
import numpy as np
import pandas as pd
import os
import re
import sklearn

In [ ]:
print(torch.__version__, torch.cuda.is_available())

In [ ]:
!which pip

os.chdir("/home/anoopkumar/Brain/HGP-SL")
!pwd

In [ ]:
# !python main_cudastreams2.py --epochs 10 --dataset ADNI_612 --batch_size 4 --lr 0.01 --dropout_ratio 0.5 --nhid 512 
# # --sparse_attention False --structure_learning False

!rm -rf /home/anoopkumar/Brain/HGP-SL/data/ADNI_612/processed
!python main.py --epochs 20 --dataset ADNI_612 --batch_size 8 --lr 0.0001 --dropout_ratio 0.5 --nhid 256 --sparse_attention False --structure_learning False

In [ ]:
#plot the times taken
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(times_labels[0:3],times_values[0:3])
ax.set_ylabel('sec')
ax.set_title('train and test')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(times_labels[3:],times_values[3:])
ax.set_ylabel('sec')
ax.set_title('train time split up')
plt.show()